In [1]:
from pandas import DataFrame, read_sql
from handler import Handler, QueryHandler, MetadataQueryHandler, MetadataUploadHandler
from sqlite3 import connect
import pandas as pd
from mashup import BasicMashup
from cultural_objects import *
import importlib


class ProcessDataQueryHandler(QueryHandler):
    def getAllActivities(self) -> DataFrame:
        with connect(self.getDbPathOrUrl()) as con:
            query_all_activities = """
            SELECT internalId, responsibleInstitute, responsiblePerson, tool, startDate, endDate, objectId, technique 
            FROM Acquisition
            UNION
            SELECT internalId, responsibleInstitute, responsiblePerson, tool, startDate, endDate, objectId, NULL AS technique 
            FROM Exporting
            UNION
            SELECT internalId, responsibleInstitute, responsiblePerson, tool, startDate, endDate, objectId, NULL AS technique 
            FROM Modelling
            UNION
            SELECT internalId, responsibleInstitute, responsiblePerson, tool, startDate, endDate, objectId, NULL AS technique 
            FROM Optimising
            ORDER BY acquisition.objectId;
            """

            df_all_activities = read_sql(query_all_activities, con)

        return df_all_activities

    # add all the other columns for queries???
    def getActivitiesByResponsibleInstitution(self, partialName: str) -> DataFrame:
        with connect(self.getDbPathOrUrl()) as con:
            query = f"""
            SELECT * 
            FROM (SELECT internalId, objectId, responsibleInstitute 
            FROM Acquisition
            UNION
            SELECT internalId, objectId, responsibleInstitute 
            FROM Exporting
            UNION
            SELECT  internalId, objectId, responsibleInstitute 
            FROM Modelling
            UNION
            SELECT internalId, objectId, responsibleInstitute 
            FROM Optimising
            UNION
            SELECT internalId, objectId, responsibleInstitute 
            FROM Processing) AS subquery
            WHERE responsibleInstitute LIKE '%{partialName}%'
            ORDER BY objectId;"""

            df = read_sql(query, con)
        return df

    # add all the other columns for queries???
    def getActivitiesByresponsiblePerson(self, partialName: str) -> DataFrame:
        with connect(self.getDbPathOrUrl()) as con:
            query = f"""SELECT * FROM 
            (SELECT internalId, objectId, responsiblePerson
            FROM Acquisition
            UNION
            SELECT internalId, objectId, responsiblePerson 
            FROM Exporting
            UNION
            SELECT  internalId, objectId, responsiblePerson 
            FROM Modelling
            UNION
            SELECT internalId, objectId, responsiblePerson 
            FROM Optimising
            UNION
            SELECT internalId, objectId, responsiblePerson 
            FROM Processing) AS subquery
            WHERE responsiblePerson LIKE '%{partialName}%'
            ORDER BY objectId;"""

            df = read_sql(query, con)
        return df

    # add all the other columns for queries???
    def getActivitiesUsingTool(self, partialName: str) -> DataFrame:
        with connect(self.getDbPathOrUrl()) as con:
            query = f"""SELECT * FROM (SELECT internalId, objectId, tool
            FROM Acquisition
            UNION
            SELECT internalId, objectId, tool 
            FROM Exporting
            UNION
            SELECT  internalId, objectId, tool 
            FROM Modelling
            UNION
            SELECT internalId, objectId, tool 
            FROM Optimising
            UNION
            SELECT internalId, objectId, tool 
            FROM Processing) AS subquery
            WHERE tool LIKE '%{partialName}%'
            ORDER BY objectId;"""

            df = read_sql(query, con)
        return df

    def getActivitiesUsingTool(self, partialName: str) -> DataFrame:
        with connect(self.getDbPathOrUrl()) as con:
            query = f"""SELECT * FROM (SELECT internalId, objectId, tool
            FROM Acquisition
            UNION
            SELECT internalId, objectId, tool 
            FROM Exporting
            UNION
            SELECT  internalId, objectId, tool 
            FROM Modelling
            UNION
            SELECT internalId, objectId, tool 
            FROM Optimising
            UNION
            SELECT internalId, objectId, tool 
            FROM Processing) AS subquery
            WHERE tool LIKE '%{partialName}%'
            ORDER BY objectId;"""

            df = read_sql(query, con)
        return df

    def getActivitiesStartedAfter(self, date: str) -> DataFrame:
        with connect(self.getDbPathOrUrl()) as con:
            query = f"""
            SELECT * FROM (SELECT internalId, objectId, startDate
            FROM Acquisition
            UNION
            SELECT internalId, objectId, startDate 
            FROM Exporting
            UNION
            SELECT  internalId, objectId, startDate
            FROM Modelling
            UNION
            SELECT internalId, objectId, startDate
            FROM Optimising
            UNION
            SELECT internalId, objectId, startDate
            FROM Processing) AS subquery
            WHERE startDate >= "{date}"
            ORDER BY objectId;"""

            df = read_sql(query, con)
        return df

    def getActivitiesEndedBefore(self, date: str) -> DataFrame:
        with connect(self.getDbPathOrUrl()) as con:
            query = f"""
            SELECT * FROM (SELECT internalId, objectId, endDate
            FROM Acquisition
            UNION
            SELECT internalId, objectId, endDate 
            FROM Exporting
            UNION
            SELECT  internalId, objectId, endDate 
            FROM Modelling
            UNION
            SELECT internalId, objectId, endDate 
            FROM Optimising
            UNION
            SELECT internalId, objectId, endDate 
            FROM Processing) AS subquery
            WHERE endDate <= "{date}"
            ORDER BY objectId;"""

            df = read_sql(query, con)
        return df

    def getAcquisitionsByTechnique(self, partialName: str) -> DataFrame:
        with connect(self.getDbPathOrUrl()) as con:
            query = f"""
            SELECT *
            FROM Acquisition
            WHERE technique LIKE "%{partialName}%"
            ORDER BY objectId
            """

            df = read_sql(query, con)
        return df

In [2]:
class BasicMashup(object):
    def __init__(self) -> None:
        self.metadataQuery = list()
        self.processQuery = list()

    def cleanMetadataHandlers(self) -> bool:  # Romolo
        self.metadataQuery = []
        return len(self.metadataQuery) == 0

    def cleanProcessHandlers(self) -> bool:  # Pietro
        self.processQuery = []
        return len(self.processQuery) == 0

    def addMetadataHandler(self, handler: MetadataQueryHandler) -> bool:  # Simone
        len_metadataQuery = len(self.metadataQuery)
        self.metadataQuery.append(handler)
        return (
            len(self.metadataQuery) == len_metadataQuery + 1
            and self.metadataQuery[-1] == handler
        )

    def addProcessHandler(self, handler: ProcessDataQueryHandler) -> bool:  # Ludovica
        len_processQuery = len(self.processQuery)
        self.processQuery.append(handler)
        return (
            len(self.processQuery) == len_processQuery + 1
            and self.processQuery[-1] == handler
        )

    def getAllCulturalHeritageObjects(self) -> list[CulturalHeritageObject]:  # Ludovica
        print(self.metadataQuery[0].getAllCulturalHeritageObjects())

In [3]:
upload = MetadataUploadHandler()
upload.setDbPathOrUrl("http://192.168.1.68:9999/blazegraph/")
upload.pushDataToDb("data/meta.csv")

metadata_qh = MetadataQueryHandler()
metadata_qh.setDbPathOrUrl("http://192.168.1.68:9999/blazegraph/")

mashup = BasicMashup()
mashup.addMetadataHandler(metadata_qh)

df_objects = metadata_qh.getAllCulturalHeritageObjects()
df_people = metadata_qh.getAllPeople().sort_values(by="id")

# merge the two dataframes

1482
1497
1523
1519
1500-1599
1556
1530
1596
1579
1758
1758
1757
1582-1585
1586
1591
1753
1500-1599
1500-1599
1500-1599
1500-1599
1500-1599
1500-1599
1500-1599
1603
1500-1599
1603
1500-1599
1500-1599
1600-1699
1642
1700-1799
1500-1599
1600-1699
1500-1599


In [4]:
df_objects.sort_values(by="id")

,uri,type,id,title,date,owner,place,author
4,https://breaking-data.github.io/Data-Science-P...,https://breaking-data.github.io/Data-Science-P...,1,Nautical chart,1482,BUB,Bologna,https://breaking-data.github.io/Data-Science-P...
16,https://breaking-data.github.io/Data-Science-P...,http://schema.org/Map,2,The History of Plants,1497,BUB,Bologna,https://breaking-data.github.io/Data-Science-P...
29,https://breaking-data.github.io/Data-Science-P...,https://breaking-data.github.io/Data-Science-P...,3,On Medical Material,1523,BUB,Bologna,https://breaking-data.github.io/Data-Science-P...
10,https://breaking-data.github.io/Data-Science-P...,http://schema.org/Map,4,The Natural History,1519,BUB,Bologna,https://breaking-data.github.io/Data-Science-P...
31,https://breaking-data.github.io/Data-Science-P...,https://breaking-data.github.io/Data-Science-P...,5,Incomplete coastal profile of the American co...,1500-1599,BUB,Bologna,https://breaking-data.github.io/Data-Science-P...
2,https://breaking-data.github.io/Data-Science-P...,https://breaking-data.github.io/Data-Science-P...,6,Map of Cusco,1556,BUB,Bologna,https://breaking-data.github.io/Data-Science-P...
1,https://breaking-data.github.io/Data-Science-P...,http://schema.org/Map,7,On the New World,1530,BUB,Bologna,https://breaking-data.github.io/Data-Science-P...
13,https://breaking-data.github.io/Data-Science-P...,http://schema.org/Map,8,Natural and Moral History of the Indies,1596,BUB,Bologna,https://breaking-data.github.io/Data-Science-P...
12,https://breaking-data.github.io/Data-Science-P...,http://schema.org/Map,9,Tobacco,1579,BUB,Bologna,https://breaking-data.github.io/Data-Science-P...
9,https://breaking-data.github.io/Data-Science-P...,https://breaking-data.github.io/Data-Science-P...,10,"Jaguar (Linnaeus, 1758)",1758,Sistema Museale di Ateneo di Bologna,Bologna,NaN


In [5]:
cultural_objects = []
module = importlib.import_module("cultural_objects")
df_objects = metadata_qh.getAllCulturalHeritageObjects().sort_values(by="id")
df_people = metadata_qh.getAllPeople().sort_values(by="id")

# merge the two dataframes
df_merged = pd.merge(
    df_objects,
    df_people.rename(
        columns={"uri": "author_uri", "name": "author_name", "id": "author_id"}
    ),
    left_on="author",
    right_on="author_uri",
    how="left",
)

for index, row in df_merged.iterrows():

    # info about the object
    ob_id = row.id
    title = row.title.strip()
    date = row.date if not pd.isna(row.date) else None
    owner = row.owner
    place = row.place

    # info about the author
    authors = set()
    # check if an author exists 
    if not pd.isna(row.author_id):
        author_id = row.author_id
        author_name = row.author_name.strip() 
        author = Person(id=author_id, name=author_name)
        authors.add(author)

    # get the subclass name for the object
    object_subclass = row.type.removeprefix(
            "https://breaking-data.github.io/Data-Science-Project/"
        ).removeprefix("http://schema.org/")

    # import the module and get the class
    subclass = getattr(module, object_subclass)
    instance = subclass(id=ob_id, title=title, date=date, owner=owner, place=place, hasAuthor=authors)

    # add the cultural object to the list
    cultural_objects.append(instance)

print(cultural_objects[1].getDate())

1497
